# DSA-110 Bandpass Calibration — PB-threshold window + field-combine

This notebook reflects the updated method:
- Fast QA ranks refant (writes `refant_ranking.json`).
- Auto-selects a contiguous FIELD window around the PB peak using a primary-beam gain threshold (`--bp-min-pb`).
- Solves K, then Bandpass/Gains with `combine='scan,field'` across the selected window (optional).

Run this with a CASA 6 kernel (casatools/casatasks available).

In [1]:
# Setup
import os, sys, json
# Be conservative with threading to avoid BLAS/OMP conflicts in CASA
os.environ.setdefault('OMP_NUM_THREADS', '1')
os.environ.setdefault('OPENBLAS_NUM_THREADS', '1')
os.environ.setdefault('MKL_NUM_THREADS', '1')
os.environ.setdefault('NUMEXPR_NUM_THREADS', '1')

sys.path.insert(0, os.path.abspath('../../src'))  # ensure dsa110_contimg is importable

# Measurement Set path (adjust as needed)
MS = '/scratch/dsa110-contimg/data-samples/ms/0834_555_transit_reconv3/2025-10-03T15:15:58.ms'

# VLA calibrator catalog
CAL_CATALOG = '/data/dsa110-contimg/data-samples/catalogs/vlacalibrators.txt'

# Auto-selection configuration
CAL_SEARCH_RADIUS_DEG = 2.0   # search radius for catalog candidates (deg)
BP_MIN_PB = 0.98              # PB gain threshold (relative to peak)
BP_WINDOW_FALLBACK = 3        # fallback window if threshold is not used
COMBINE_FIELDS = True         # combine fields for BP/G solves
DO_FLAGGING = False           # pre-solve flagging (set True if needed)

# QA output directory
QA_DIR = '/tmp/qa-auto'
os.makedirs(QA_DIR, exist_ok=True)

MS

'/scratch/dsa110-contimg/data-samples/ms/0834_555_transit_reconv3/2025-10-03T15:15:58.ms'

In [2]:
# 1) Fast QA to rank refant
from dsa110_contimg.qa.fast_plots import run_fast_plots

arts = run_fast_plots(MS, output_dir=QA_DIR)
print('Fast QA artifacts:', arts)

with open(os.path.join(QA_DIR, 'refant_ranking.json'), 'r', encoding='utf-8') as f:
    refant = json.load(f)['recommended']['antenna_id']
print('Recommended refant:', refant)
refant

Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit_reconv3/2025-10-03T15:15:58.ms: 25 columns, 111744 rows
Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit_reconv3/2025-10-03T15:15:58.ms::DATA_DESCRIPTION: 3 columns, 1 rows
Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit_reconv3/2025-10-03T15:15:58.ms::SPECTRAL_WINDOW: 14 columns, 1 rows
Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit_reconv3/2025-10-03T15:15:58.ms::ANTENNA: 8 columns, 117 rows
Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit_reconv3/2025-10-03T15:15:58.ms::ANTENNA: 8 columns, 117 rows
Fast QA artifacts: ['amp_vs_time_fast.png', 'amp_vs_freq_fast.png', 'phase_vs_freq_fast.png', 'uv_plane_fast.png', 'per_antenna_metrics_fast.png', 'phase_si

67

In [3]:
# 2) Catalog-based field selection with PB threshold
from dsa110_contimg.calibration.selection import select_bandpass_from_catalog

sel_str, indices, wflux, calinfo = select_bandpass_from_catalog(
    MS, CAL_CATALOG, search_radius_deg=CAL_SEARCH_RADIUS_DEG,
    window=BP_WINDOW_FALLBACK, min_pb=BP_MIN_PB
)
name, ra_deg, dec_deg, flux_jy = calinfo
print(f'Calibrator: {name} (RA {ra_deg:.4f} deg, Dec {dec_deg:.4f} deg, flux {flux_jy:.3f} Jy)')
print('Selected fields:', sel_str, 'indices:', indices)

# Quick plot of PB-weighted flux per field
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(8,3))
plt.plot(np.arange(len(wflux)), wflux, '-o', ms=3)
plt.axvspan(indices[0]-0.5, indices[-1]+0.5, color='orange', alpha=0.2, label='selected')
plt.xlabel('Field index')
plt.ylabel('PB-weighted flux (Jy)')
plt.title('Bandpass field auto-selection (PB threshold)')
plt.legend()
plt.tight_layout()
plt.show()

sel_str

Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit_reconv3/2025-10-03T15:15:58.ms::FIELD: 9 columns, 24 rows
Calibrator: 0824+558 (RA 126.1968 deg, Dec 55.8785 deg, flux 1.200 Jy)
Selected fields: 17~23 indices: [17, 18, 19, 20, 21, 22, 23]


/tmp/ipykernel_62521/527142941.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


'17~23'

In [4]:
# 3) Solve K, then BP/G (optionally combining fields)
from importlib.util import find_spec
if find_spec('casatasks') is None or find_spec('casatools') is None:
    raise RuntimeError('CASA not available in this kernel. Switch to a CASA 6 kernel.')

from dsa110_contimg.calibration.calibration import solve_delay, solve_bandpass, solve_gains
from dsa110_contimg.calibration.flagging import reset_flags, flag_zeros, flag_rfi

if DO_FLAGGING:
    reset_flags(MS); flag_zeros(MS); flag_rfi(MS)

print('Solving delay (K) ...')
ktabs = solve_delay(MS, sel_str, str(refant))
print('K tables:', ktabs)

print('Solving bandpass (B) ... combine_fields =', COMBINE_FIELDS)
bptabs = solve_bandpass(MS, sel_str, str(refant), ktabs[0], combine_fields=COMBINE_FIELDS)
print('BP tables:', bptabs)

print('Solving gains (G) ... combine_fields =', COMBINE_FIELDS)
gtabs = solve_gains(MS, sel_str, str(refant), ktabs[0], bptabs, combine_fields=COMBINE_FIELDS)
print('G tables:', gtabs)

tables = ktabs[:1] + bptabs + gtabs
print('All tables:')
for t in tables:
    print(' -', t)
tables


Solving delay (K) ...


: 

### CLI equivalent

```bash
PYTHONPATH=src:$PYTHONPATH /opt/miniforge/envs/casa6/bin/python -m dsa110_contimg.calibration.cli calibrate \
  --ms /scratch/dsa110-contimg/data-samples/ms/0834_555_transit_reconv3/2025-10-03T15:15:58.ms \
  --auto-fields \
  --cal-catalog /data/dsa110-contimg/data-samples/catalogs/vlacalibrators.txt \
  --cal-search-radius-deg 2.0 \
  --bp-min-pb 0.98 \
  --refant-ranking /tmp/qa-auto/refant_ranking.json \
  --bp-combine-field
```